In [1]:
import pymysql
import sys
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
import matplotlib.pyplot as plt
# pyqt5를 지원하는 matplotlib 모듈 호출
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas

import pandas as pd
from pandas import Series, DataFrame

In [2]:
class MyWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.setupUI()

    def setupUI(self):
        self.setGeometry(600, 200, 1200, 600)
        self.setWindowTitle("Usn Chart Viewer v0.1")
        self.setWindowIcon(QIcon('icon.png'))
        
        groupBox = QGroupBox("서버 아이피 주소 입력", self)
        groupBox.resize(280, 80)
        self.lineEdit = QLineEdit()
        self.pushButton = QPushButton("차트 그리기")
        self.pushButton.clicked.connect(self.pushButtonClicked)   
        
        self.tableWidget = QTableWidget(self)
        self.tableWidget.setRowCount(200) # seq, node , temp, humi, date, time
        self.tableWidget.setColumnCount(6)
        self.tableWidget.setEditTriggers(QAbstractItemView.NoEditTriggers)
        
        
        # FigureCanvas 객체를 생성한다.
        self.fig = plt.Figure()
        self.canvas = FigureCanvas(self.fig)
        
        self.fig_2 = plt.Figure()
        self.canvas_2 = FigureCanvas(self.fig_2)

        leftLayout = QVBoxLayout()
        leftLayout.addWidget(self.canvas)
        leftLayout.addWidget(self.canvas_2)

        # Right Layout
        rightLayout = QVBoxLayout()
        rightLayout.addWidget(groupBox)
        rightLayout.addWidget(self.lineEdit)
        rightLayout.addWidget(self.pushButton)
        rightLayout.addWidget(self.tableWidget)
        rightLayout.addStretch(1) # 크기 조절이 가능한 공백을 추가

        layout = QHBoxLayout()
        layout.addLayout(leftLayout)
        layout.addLayout(rightLayout)
        layout.setStretchFactor(leftLayout, 1)
        layout.setStretchFactor(rightLayout, 0)

        self.setLayout(layout)

    def pushButtonClicked(self):
        
        host_ip = self.lineEdit.text()
        conn = pymysql.connect(host= host_ip, user='root', password='1234', db='usn', charset='utf8')
        df = pd.read_sql('select * from usnnode', con = conn)
        temp_plot = self.fig.add_subplot(211)
        temp_plot.plot(df.index, df['temp'],label='temp')
        temp_plot.legend(loc='upper right')
        humi_plot = self.fig_2.add_subplot(212)
        humi_plot.plot(df.index, df['humi'],label='humi')
        humi_plot.legend(loc='upper right')
        temp_plot.grid()
        humi_plot.grid()
        
        self.canvas.draw()
        self.canvas_2.draw()
        
        self.setTableWidgetData()
        
    def setTableWidgetData(self):
        column_idx_lookup = {'seq': 0, 'node': 1, 'temp': 2, 'humi' : 3, 'date' : 4, 'time':5}
        column_headers = ['seq', 'node', 'temp', 'humi', 'date', 'temp']
        self.tableWidget.setHorizontalHeaderLabels(column_headers)
        conn = pymysql.connect(host= 'localhost', user='root', password='1234', db='usn', charset='utf8')
        curs = conn.cursor(pymysql.cursors.DictCursor)
        curs.execute("""select * from usnnode""")
        rows = curs.fetchall()
        seq_list = []
        node_list = []
        temp_list = []
        humi_list = []
        date_list = []
        time_list = []
        db_dict = {}
        length = len(rows)
        for i in range(length):
            seq = rows[i]['seq']
            node = rows[i]['node']
            temp = rows[i]['temp']
            humi = rows[i]['humi']
            db_date = rows[i]['date']
            db_time = rows[i]['time']
            seq_list.append(seq)
            node_list.append(node)
            temp_list.append(temp)
            humi_list.append(humi)
            date_list.append(db_date)
            time_list.append(db_time)

        db_dict['seq'] = seq_list
        db_dict['node'] = node_list
        db_dict['temp'] = temp_list
        db_dict['humi'] = humi_list
        db_dict['date'] = date_list
        db_dict['time'] = time_list
        for k, v in db_dict.items():
            col = column_idx_lookup[k]
            for row, val in enumerate(v):
                item = QTableWidgetItem(val)
                self.tableWidget.setItem(row, col, item)
                
        self.tableWidget.resizeColumnsToContents()
        self.tableWidget.resizeRowsToContents()

In [3]:
if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = MyWindow()
    window.show()
    app.exec_()